In [1]:
import tensorflow as tf

/Users/jiazhuo/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jiazhuo/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jiazhuo/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jiazhuo/o

In [ ]:
#创建
files=tf.train.match_filenames_once("")
#一般设置为True
filename_queue=tf.train.string_input_producer(files,shuffle=False)


reader=tf.TFRecordReader()
_,serialized_example=reader.read(filename_queue)
features=tf.parse_single_example(serialized_example,
                                 features={
                                     'image':tf.FixedLenFeature([],tf.string),
                                     'lable':tf.FixedLenFeature([],tf.int64),
                                     'height':tf.FixedLenFeature([],tf.int64),
                                     'width':tf.FixedLenFeature([],tf.int64),
                                     'channels':tf.FixedLenFeature([],tf.int64),
                                 })
image,lable=features['image'],features['lable']
height,width=features['height'],features['width']
channels=features['channels']

#
decoded_image=tf.decode_raw(image,tf.uint8)
decoded_image.set_shape([height,width,channels])

image_size=299
distored_image=preprocess_for_train(decoded_image,image_size,image_size,None)

min_after_dequeue=10000
batch_size=100
capacity=min_after_dequeue+3*batch_size
image_batch,lable_batch=tf.train.shuffle_batch([distored_image,lable],batch_size=batch_size
                                               capacity=capacity,min_after_dequeue=min_after_dequeue)


learning_rate=0.01

#forward   前向传播
logit=forward(image_batch)

loss=caculte_loss(logit,lable_batch)
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)


with tf.Session() as sess:
    sess.run((tf.global_variables_initializer(),tf.local_variables_initializer))
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)

    TRAINING_ROUNDS=5000
    for i in range(TRAINING_ROUNDS):
        sess.run(train_step)

    coord.request_stop()
    coord.join(threads)